In [10]:
from IPython.core.display import display
import pyodbc
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori,association_rules, fpmax, fpgrowth
from Query import Query

from configuration import Configuration
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', 0)

conn = pyodbc.connect(Configuration.getConfigValue('database_connection'))
supportStr = Configuration.getConfigValue('QUESTION3_METRIC1_SUPORT')
support = float(supportStr)
query = Query.getQuery('query_3_1')
result = pd.read_sql_query(query,conn)
dataframes = pd.DataFrame(result)

if(len(dataframes) == 0):
    display("Não há dados para analisar")
else:
    dataframes = pd.pivot_table(dataframes,  index="ID_COMMIT", columns=['RULE_DESCRIPTION'],values="VALOR")
    dataset = []
    for i in range(0, len(dataframes)):
        datasetI = []
        for j in range(0, len(dataframes.columns)):
            if str(dataframes.values[i,j]) == "1.0"  :
                datasetI.append( str(dataframes.columns[j]))
        dataset.append(datasetI)
        datasetI = []

    te = TransactionEncoder()
    te_ary = te.fit(dataset).transform(dataset)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = fpgrowth(df, min_support=support, use_colnames=True)
    display(frequent_itemsets)
    rules = association_rules(frequent_itemsets, metric="support", min_threshold=support)
    display(rules)


,support,itemsets
0,0.560645,(S1192 - String literals should not be duplicated)
1,0.454623,(S121 - Control structures should use curly braces)
2,0.185666,"(S121 - Control structures should use curly braces, S1192 - String literals should not be duplicated)"


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(S121 - Control structures should use curly braces),(S1192 - String literals should not be duplicated),0.454623,0.560645,0.185666,0.408396,0.728439,-0.069216,0.742651
1,(S1192 - String literals should not be duplicated),(S121 - Control structures should use curly braces),0.560645,0.454623,0.185666,0.331165,0.728439,-0.069216,0.815414
